In [1]:
%pip install requests

  Using cached requests-2.32.3-py3-none-any.whl.metadata (4.6 kB)
  Using cached charset_normalizer-3.3.2-cp312-cp312-win_amd64.whl.metadata (34 kB)
  Using cached urllib3-2.2.3-py3-none-any.whl.metadata (6.5 kB)
  Using cached certifi-2024.8.30-py3-none-any.whl.metadata (2.2 kB)
Using cached requests-2.32.3-py3-none-any.whl (64 kB)
Using cached certifi-2024.8.30-py3-none-any.whl (167 kB)
Using cached charset_normalizer-3.3.2-cp312-cp312-win_amd64.whl (100 kB)
Using cached urllib3-2.2.3-py3-none-any.whl (126 kB)
Note: you may need to restart the kernel to use updated packages.


In [2]:
import requests
import sys
import json
from abc import ABC, abstractmethod
import re
import logging

In [70]:
class LLMInterface(ABC):
    @abstractmethod
    def create_response(self, input):
        pass
    

class ResponseGeneratorInterface(ABC):
    @abstractmethod
    def generate_response(self, input, llm:LLMInterface):
        pass

class OllamaManagerInterface(ABC):
    @abstractmethod
    def list_models(self):
        pass
        
    @abstractmethod
    def get_model_by_index(self, index):
        pass
    

class ChatInterface(ABC):
    @abstractmethod
    def manage_session(self):
        pass

    @abstractmethod
    def get_user_input(self):
        pass

    @abstractmethod
    def give_output(self,user_input):
        pass

class DatabaseInterface(ABC):
    @abstractmethod
    def __init__(self, *args, **kwargs):
        pass

    @abstractmethod
    def add(self, messages):
        pass

    @abstractmethod
    def get(self):
        pass

    @abstractmethod
    def delete(self):
        pass

In [71]:
"""
Conversation History class that manages the history of messages in a conversation.

This class provides methods to add, retrieve, and delete the conversation history.
"""
class ConversationHistory(DatabaseInterface):
    """
    Manages the history of messages in a conversation.

    Attributes:
        history (list): A list of message dictionaries, where each dictionary contains the keys 'role' and 'message'.
    """
    def __init__(self):
        self.history = []

    def add(self, messages):
        """
        Adds a new set of messages to the conversation history.

        Args:
            messages (dict): A dictionary containing the 'role' and 'message' keys.
        """
        self.history.append(messages)

    def get(self):
        """
        Retrieves the entire conversation history.

        Returns:
            list: A list of message dictionaries, where each dictionary contains the 'role' and 'message' keys.
        """
        return self.history

    def delete(self):
        """
        Deletes the entire conversation history.
        """
        self.history = []

"""
ChatLLM class that manages the interaction with a language model.

This class is responsible for creating responses based on the conversation history and the input from the user.
"""
class ChatLLM(LLMInterface):
    """
    Manages the interaction with a language model.

    Args:
        model (str): The name or identifier of the language model to use.
        response_generator (ResponseGeneratorInterface): An object that generates the response from the language model.
        conversation_history (DatabaseInterface): An object that manages the conversation history.
    """
    def __init__(
        self,
        model:str,
        response_generator : ResponseGeneratorInterface,
        conversation_history: DatabaseInterface
    ):
        self.model = model
        self._response_generator = response_generator
        self.history = conversation_history

    def create_response(self, input:str):
        """
        Creates a response to the user's input based on the conversation history.

        Args:
            input (str): The user's input.

        Yields:
            str: The generated response, chunk by chunk.
        """
        total_response = ""
        prompt = self._prepare_prompt(input, self.history)
        for chunk in self._response_generator.generate_response(prompt,self):
            total_response += chunk
            yield chunk
        
        self.history.add({"role": 'human', 'message':input})
        self.history.add({"role": 'ai', 'message':total_response})

    def _prepare_prompt(self, input, llm_history):
        """
        Prepares the prompt for the language model based on the conversation history.

        Args:
            input (str): The user's input.
            llm_history (DatabaseInterface): The conversation history.

        Returns:
            str: The prepared prompt.
        """
        prompt = "\n".join(
            [
                f"{entry['role']}: {entry['message']}"
                for entry in llm_history.get()
            ]
        )
        prompt += f"\nHuman: {input}\nAI:"
        return prompt
    



In [72]:
class ChatResponseGenerator(ResponseGeneratorInterface):
    def __init__(self,base_url:str):
        """
        Initializes the ChatResponseGenerator with the specified base URL.
        
        Args:
            base_url (str): The base URL for the API endpoint.
        """
        self._base_url = base_url

    def generate_response(self, input, llm:LLMInterface):
        """
    Generates a response from the specified language model by sending a request to the API endpoint.
    
    Args:
        input (str): The input text to be processed by the language model.
        llm (LLMInterface): The language model interface to use for generating the response.
    
    Yields:
        str: The generated response, streamed in chunks.
    
    Raises:
        RuntimeError: If there is an error communicating with the server or a connection error.
        """
        _url = f"{self._base_url}/api/generate"
        payload = {"model": llm.model, "prompt": input, "stream": True}
        try:

            response = requests.post(_url, json=payload, stream=True)
            response.raise_for_status()

            for line in response.iter_lines():
                if line:
                    chunk = json.loads(line)
                    if "response" in chunk:
                        yield chunk["response"]

        except requests.HTTPError as e:
            raise RuntimeError(f"Error communicating with server: {e}") from e  
        except requests.RequestException as e:
            raise RuntimeError(f"Connection error: {e}") from e
        

class UserChatInterface(ChatInterface):
    def __init__(self,ollama_manager:OllamaManagerInterface,response_generator:ResponseGeneratorInterface):
        """
        Initializes the object
        
        Args:
            ollama_manager (OllamaManagerInterface): The OllamaManagerInterface subclass instance to use for managing the language models.
            response_generator (ResponseGeneratorInterface): The ResponseGeneratorInterface subclass instance to use for generating responses.
        """
        self._ollama_manager = ollama_manager
        self._response_generator = response_generator

    def give_role(self,role_prompt):
        """
        Assigning the role to the model by adding the role prompt to the conversation history.
        
        Args:
            role_prompt (str): The prompt to be added to the conversation history as a message from the human.
        """
        self._llm.history.delete()
        self._llm.history.add({'role':'human','message':role_prompt})

    def _user_input_for_model(self):
        """
        Asks the user to choose a model from the available models and initializes the ChatLLM object.
        """
        print("Available models:")
        self._ollama_manager.list_models()
        while True:
            try:
                model_index = int(input("Please choose the index of the model you want to use: "))
                self.initialize_llm(model_index)
                break  # Exit loop if input is valid
            except ValueError:
                print("Invalid input. Please enter a valid integer.")

    def initialize_llm(self,model_index:int,conversation_history=ConversationHistory()):
        self._llm = ChatLLM(self._ollama_manager.get_model_by_index(model_index)['model'],self._response_generator,conversation_history)
    
    def manage_session(self):
        self._user_input_for_model()
        while True:
            user_input = self.get_user_input()
            if self._should_exit(user_input):
                break
            self.give_output(user_input)


    def get_user_input(self):
        user_input = input("You: ")
        print(f"You: {user_input}")
        return user_input

    def _should_exit(self, user_input):
        return user_input.lower() in ["exit", "quit", "bye"]


    def give_output(self, user_input):
        print(f"Chatbot: ", end="")
        output = self._llm.create_response(user_input)
        for line in output:
            print(line, end="", flush=True)
        print()
    
class OllamaManager(OllamaManagerInterface):
    def __init__(self, base_url: str):
        self.base_url = base_url
        self.available_models = []

    def list_models(self):
        if not self.available_models:
            self._get_all_models()
        for model in self.available_models:
            print(model)

    def get_model_by_index(self,index:int):
        if not self.available_models:
            self._get_all_models()
        assert index < len(self.available_models), "Index out of range"
        return self.available_models[index]

    def _get_all_models(self):
        if not self.available_models:  # Check if the cache is empty
            url = f"{self.base_url}/api/tags"
            try:
                response = requests.get(url)
                response.raise_for_status()
                self.available_models = response.json()["models"]
            except requests.RequestException as e:
                print(f"Error: Could not fetch models from Ollama at {self.base_url}")
                print(f"Error: Could not connect to Ollama at {self.base_url}")
                print(
                    f"Please ensure Ollama is running and accessible. Error details: {str(e)}"
                )
                sys.exit(1)
        return self.available_models



In [73]:
ollama_manager = OllamaManager(base_url= "http://localhost:11434")
response_generator = ChatResponseGenerator(base_url="http://localhost:11434")

In [74]:
user_interface = UserChatInterface(ollama_manager,response_generator)
user_interface.manage_session()

Available models:
{'name': 'llama3.1:latest', 'model': 'llama3.1:latest', 'modified_at': '2024-09-25T19:29:51.3062143+05:30', 'size': 4661230766, 'digest': '42182419e9508c30c4b1fe55015f06b65f4ca4b9e28a744be55008d21998a093', 'details': {'parent_model': '', 'format': 'gguf', 'family': 'llama', 'families': ['llama'], 'parameter_size': '8.0B', 'quantization_level': 'Q4_0'}}
You: Hi
Chatbot: Hello! How can I assist you today?
You: Bye


In [75]:
role_prompt = """
You run in a loop of Thought, Action, PAUSE, Observation.
At the end of the loop you output an Answer
Use Thought to describe your thoughts about the question you have been asked.
Use Action to run one of the actions available to you - then return PAUSE.
Observation will be the result of running those actions.
You must not take two or more actions at a particular time. The Action line must have the format: Action: action_name: argument

Your available actions are:

calculate:
e.g. calculate: 4 * 7 / 3
Runs a calculation and returns the number - uses Python so be sure to use floating point syntax if necessary

average_dog_weight:
e.g. average_dog_weight: Collie
returns average weight of a dog when given the breed


Example session:

Question: How much does a Bulldog weigh?
Thought: I should look the dogs weight using average_dog_weight
Action: average_dog_weight: Bulldog
PAUSE

You will be called again with this:

Observation: A Bulldog weights 51 lbs

You then output:

Answer: A bulldog weights 51 lbs

Note: Don't give any output to this response.
""".strip()

In [76]:
llm = ChatLLM(ollama_manager.get_model_by_index(0)['model'],response_generator,ConversationHistory())
llm.history.add({'role':'human','message':role_prompt})

In [77]:
action_re = re.compile(r'^Action: (\w+): (.*)')   # python regular expression to selection action

In [78]:
def calculate(what):
    return eval(what)

def average_dog_weight(name):
    if name in "Scottish Terrier": 
        return("Scottish Terriers average 20 lbs")
    elif name in "Border Collie":
        return("a Border Collies average weight is 37 lbs")
    elif name in "Toy Poodle":
        return("a toy poodles average weight is 7 lbs")
    else:
        return("An average dog weights 50 lbs")

known_actions = {
    "calculate": calculate,
    "average_dog_weight": average_dog_weight
}

In [79]:
def query(question,agent, max_turns=5):
    i = 0
    next_prompt = question
    while i < max_turns:
        i += 1
        result = ""
        for token in agent.create_response(next_prompt):
            result += token
            print(token,end="")
        print()
        actions = [
            action_re.match(a) 
            for a in result.split('\n') 
            if action_re.match(a)
        ]
        if actions:
            next_prompt = ""
            for action in actions:
                action_type, action_input = action.groups()
                if action_type not in known_actions:
                    raise Exception("Unknown action: {}: {}".format(action_type, action_input))
                print(" -- running {} {}".format(action_type, action_input))
                observation = known_actions[action_type](action_input)
                print("Observation:", observation)
                next_prompt += f"Observation: {observation}\n"
        else:
            return

In [80]:
question = """I have 2 dogs, a border collie and a scottish terrier. What is their combined weight"""
query(question,llm)

Thought: To find the combined weight of the two dogs, I need to get the weights of a Border Collie and a Scottish Terrier individually and then add them together.

Action: average_dog_weight: Border Collie
 -- running average_dog_weight Border Collie
Observation: a Border Collies average weight is 37 lbs
Thought: Now that I have one dog's weight, I need to get the weight of the other dog, the Scottish Terrier, and then add it to the first one.

Action: average_dog_weight: Scottish Terrier
 -- running average_dog_weight Scottish Terrier
Observation: Scottish Terriers average 20 lbs
Thought: Now that I have both dogs' weights, I can simply add them together to get their combined weight. I just need to run a calculation